In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X = np.ones((10000,1000, 100))
Y = np.concatenate((np.ones(8000), np.zeros(2000)))

In [3]:
X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [6]:
print(X_2.shape)

(2000, 1000, 100)


In [18]:
print(X_1.shape)

(8000, 1000, 100)


In [129]:
print(sum(Y_1 == 0))

1600


In [66]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print('train -  {}   |   test -  {}'.format(
        np.bincount(y[train]), np.bincount(y[test])))
    print(y[train])
    
# kf = KFold(n_splits=3)
# for train, test in kf.split(X, y):
#     print('train -  {}   |   test -  {}'.format(
#         np.bincount(y[train]), np.bincount(y[test])))

train -  [30  3]   |   test -  [15  2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
train -  [30  3]   |   test -  [15  2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
train -  [30  4]   |   test -  [15  1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]


In [2]:
import torch 

In [6]:
a = torch.ones((3,4,5,6))

In [12]:
a.mean(dim=2).shape

torch.Size([3, 4, 6])

In [3]:
import tensorflow as tf

def assert_shape(tensor, expected_shape):
    tensor_shape = tensor.get_shape().as_list()
    error_message = "tensor {name} shape {actual} != {expected}"
    assert tensor_shape == expected_shape, error_message.format(
        name=tensor.name, actual=tensor_shape, expected=expected_shape)

2025-04-09 22:45:41.841179: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 22:45:41.942105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 22:45:41.979827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 22:45:41.991824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 22:45:42.070210: I tensorflow/core/platform/cpu_feature_guar

In [9]:
a = tf.constant([1,2,3])

In [10]:
a

TypeError: 

In [11]:
print(a)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [13]:
assert_shape(a , [3])

In [1]:
import torch
from torch.nn import MSELoss

a = torch.tensor([1.0], requires_grad=True)

y = a * 2
z = a * 4

In [2]:
def symmetric_mse_loss(input1, input2):
    """Like F.mse_loss but sends gradients to both directions

    Note:
    - Returns the sum over all examples. Divide by the batch size afterwards
      if you want the mean.
    - Sends gradients to both input1 and input2.
    """
    # assert input1.size() == input2.size()
    # num_classes = input1.size()[1]
    return torch.sum((input1 - input2)**2) 

In [3]:
y.retain_grad()
z.retain_grad()

In [4]:
from torch.nn import functional as F
# mse = MSELoss()


c = symmetric_mse_loss(z, torch.tensor([2.0]))
d = symmetric_mse_loss(y, torch.tensor([4.0]))

In [5]:
e = c + d

In [6]:
e

tensor(8., grad_fn=<AddBackward0>)

In [7]:
e.backward()

In [8]:
y.grad

tensor([-4.])

In [9]:
z.grad

tensor([4.])

In [10]:
a.grad

tensor([8.])